# Pyomo LP
Anaconda command prompt on Windows 10:

In [191]:
# DEBUG
import pandas as pd
import numpy as np
import pyomo.kernel as pmo
import os
os.chdir("C:/Users/Admin/anaconda3/pkgs/ipopt-3.11.1-2/Library/bin/")
from pyomo.environ import (
    ConcreteModel,
    Param,
    Var,
    PositiveReals,
    Objective,
    Constraint,
    minimize,
    SolverFactory,
)

# Create the concrete model
m = ConcreteModel()
# Parameters
m.M = 3; # Number of machines
m.P = 4; # Number of personnel
m.T = 5; # Number of tasks
m.c = [2.1, 3.0, 1.6]; # Cost of using each machine m per unit time
m.e = [12.1, 13.0, 14.0, 11.6]; # Cost of employing each person p per unit time
m.o = 123.2; # Fixed overhead cost
m.u = [222.1, 223.0, 1111.6]; # Usage limit (max time over the period) for each machine m
m.v = [140.0, 141.1, 139.6, 142.9]; # Work hours limit (max time over the period) for each person p
m.l = [4.02, 5.1, 3.6, 4.29, 6.08]; # Duration of each task t in hours. Assumed to be independent on machine m!
m.d = [124.02, 125.1, 123.6, 124.29, 126.08]; # Deadline of each task t in hours
m.r = [123.02, 124.1, 122.6, 123.29, 125.08]; # Tardiness starts at these times for each task t
m.wc = 1.0; # Weight of cost objective.
m.wt = 1.0; # Weight of tardiness objective.

# Indexes
m.m = m.RangeSet(0, m.M-1) # Index of machine
m.p = m.RangeSet(0, m.P-1) # Index of personnel
m.t = m.RangeSet(0, m.T-1) # Index of personnel

# Decision variables
m.x = m.Var(m.m, within=PositiveReals, initialize=np.zeros(m.P)) # Time each machine m is used
m.y = m.Var(m.p, within=PositiveReals, initialize=np.zeros(m.P)) # Time each person p is employed
m.a = m.Var(m.t, m.m, domain=pmo.Binary); # Assignment of each task t to each machine m
m.b = m.Var(m.t, m.p, domain=pmo.Binary); # Assignment of each task t to each person m
m.f = m.Var(m.t, within=PositiveReals, initialize=np.zeros(m.T)) # Finish time for each task t

# Objective 1: Minimize the total cost of quality control resource allocation.
# Objective 2: Minimize the total tardiness.
def ObjR ule(m):
    S = 0
    for m in range(M):
        S += m.c[m] * m.x[m]
    for p in range(P):
        S += m.e[p] * m.y[p]
    S += m.o
    S *= m.wc
    
    S2 = 0
    for t in range(T):
        S2 += max(0, m.f[t] - m.r[t])
    S2 *= m.wt
    return S + S2

# No throughput objective. Assuming all tasks must be completed!
m.obj = m.Objective(rule=ObjRule, sense=minimize)

# Constraints

# Each machine has a usage limit:
def machine_usage_limit(m):
    return x(m) <= u(m) 
m.cnstr_mu = m.Constraint(m.m, rule=machine_usage_limit)

# Each person has a work hours limit:
def work_hours_limit(p):
    return y(p) <= v(p)
m.cnstr_pu = m.Constraint(m.p, rule=work_hours_limit)

In [192]:
solver = SolverFactory("ipopt.exe")
solver.solve(m)

ValueError: No variables appear in the Pyomo model constraints or objective. This is not supported by the NL file interface

In [10]:
#  ___________________________________________________________________________
#
#  Pyomo: Python Optimization Modeling Objects
#  Copyright (c) 2008-2022
#  National Technology and Engineering Solutions of Sandia, LLC
#  Under the terms of Contract DE-NA0003525 with National Technology and
#  Engineering Solutions of Sandia, LLC, the U.S. Government retains certain
#  rights in this software.
#  This software is distributed under the 3-clause BSD License.
#  ___________________________________________________________________________
#  Author: Yuriy Sereda
#  Creation Date: Dec 10, 2023 at 12:42:40 PM
"""
Optimal Resource Allocation for Quality Control using Multiobjective Multichoice Goal Programming
"""
import pandas as pd
import numpy as np
import pyomo.kernel as pmo
import os
os.chdir("C:/Users/Admin/anaconda3/pkgs/ipopt-3.11.1-2/Library/bin/")
from pyomo.environ import (
    ConcreteModel,
    Param,
    Var,
    PositiveReals,
    Objective,
    Constraint,
    minimize,
    SolverFactory,
)


def alloc_model(data):
    # Create the concrete model
    m = ConcreteModel()

    # Parameters
    M = 3; # Number of machines
    P = 4; # Number of personnel
    T = 5; # Number of tasks
    m.c = [2.1, 3.0, 1.6]; # Cost of using each machine m per unit time
    m.e = [12.1, 13.0, 14.0, 11.6]; # Cost of employing each person p per unit time
    m.o = 123.2; # Fixed overhead cost
    m.u = [222.1, 223.0, 1111.6]; # Usage limit (max time over the period) for each machine m
    m.v = [140.0, 141.1, 139.6, 142.9]; # Work hours limit (max time over the period) for each person p
    m.l = [4.02, 5.1, 3.6, 4.29, 6.08]; # Duration of each task t in hours. Assumed to be independent on machine m!
    m.d = [124.02, 125.1, 123.6, 124.29, 126.08]; # Deadline of each task t in hours
    m.r = [123.02, 124.1, 122.6, 123.29, 125.08]; # Tardiness starts at these times for each task t
    m.wc = 1.0; # Weight of cost objective.
    m.wt = 1.0; # Weight of tardiness objective.

    # Indexes
    m.m = m.RangeSet(0, M-1) # Index of machine
    m.p = m.RangeSet(0, P-1) # Index of personnel
    m.t = m.RangeSet(0, T-1) # Index of personnel
    
    # Decision variables
    m.x = m.Var(m.m, within=PositiveReals, initialize=np.zeros(P)) # Time each machine m is used
    m.y = m.Var(m.p, within=PositiveReals, initialize=np.zeros(P)) # Time each person p is employed
    m.a = m.Var(m.t, m.m, domain=pmo.Binary); # Assignment of each task t to each machine m
    m.b = m.Var(m.t, m.p, domain=pmo.Binary); # Assignment of each task t to each person m
    m.f = m.Var(m.t, within=PositiveReals, initialize=np.zeros(T)) # Finish time for each task t
    
    # Objective 1: Minimize the total cost of quality control resource allocation.
    # Objective 2: Minimize the total tardiness.
    def ObjRule(m):
        S = 0
        for m in range(M):
            S += m.c[m] * m.x[m]
        for p in range(P):
            S += m.e[p] * m.y[p]
        S += m.o
        S *= m.wc
        
        S2 = 0
        for t in range(T):
            S2 += max(0, m.f[t] - m.r[t])
        S2 *= m.wt
        return S + S2
    
    # No throughput objective. Assuming all tasks must be completed!
    m.obj = m.Objective(rule=ObjRule, sense=minimize)
    
    # Constraints
    
    # Each machine has a usage limit:
    def machine_usage_limit(m):
        return x(m) <= u(m) 
    m.cnstr_mu = m.Constraint(m.m, rule=machine_usage_limit)
    
    # Each person has a work hours limit:
    def work_hours_limit(p):
        return y(p) <= v(p)
    m.cnstr_pu = m.Constraint(m.p, rule=work_hours_limit)



    return model


def main():
    # For a range of sv values, return ca, cb, cc, and cd
    results = []
    sv_values = [1.0 + v * 1 for v in range(1, 20)]
    caf = 10000
    for sv in sv_values:
        model = reactor_design_model(pd.DataFrame(data={"caf": [caf], "sv": [sv]}))
        
        solver = SolverFactory("ipopt.exe")
        solver.solve(model)
        results.append([sv, caf, model.ca(), model.cb(), model.cc(), model.cd()])

    results = pd.DataFrame(results, columns=["sv", "caf", "ca", "cb", "cc", "cd"])
    print(results)


if __name__ == "__main__":
    main()

      sv    caf           ca           cb          cc           cd
0    2.0  10000  4585.298524  1042.113301  868.427751  1752.080212
1    3.0  10000  5343.353866   954.170333  530.094630  1586.190586
2    4.0  10000  5886.304655   865.633038  360.680432  1443.690937
3    5.0  10000  6301.993223   787.749153  262.583051  1323.837286
4    6.0  10000  6633.794005   721.064566  200.295713  1222.422858
5    7.0  10000  6906.433207   664.080116  158.114313  1135.686182
6    8.0  10000  7135.350132   615.116391  128.149248  1060.692115
7    9.0  10000  7330.817467   572.720115  106.059280   995.201569
8   10.0  10000  7500.000000   535.714286   89.285714   937.500000
9   11.0  10000  7648.080636   503.163200   76.236848   886.259658
10  12.0  10000  7778.919697   474.324372   65.878385   840.438773
11  13.0  10000  7895.462119   448.605802   57.513564   799.209257
12  14.0  10000  8000.000000   425.531915   50.658561   761.904762
13  15.0  10000  8094.348007   404.717400   44.968600   727.98

In [515]:
%reset -f
from pyomo.environ import *
import numpy as np

# create a model instance
model = ConcreteModel()

# Parameters
#Sets
M = 3; # Number of machines
P = 4; # Number of personnel
T = 5; # Number of tasks
model.m = RangeSet(0, M-1)
#model.m.pprint()
model.p = RangeSet(0, P-1)
model.t = RangeSet(0, T-1)
#Machine parameters
model.c = Param(model.m, initialize=[2.1, 3.0, 1.6]) # Cost of using each machine m per unit time
#Personnel parameters
model.e = Param(model.p, initialize=[12.1, 13.0, 14.0, 11.6]) # Cost of employing each person p per unit time
model.o = Param(initialize=123.2) # Fixed overhead cost
model.u = Param(model.m, initialize=[222.1, 223.0, 1111.6]) # Usage limit (max time over the period) for each machine m
model.v = Param(model.p, initialize=[140.0, 141.1, 139.6, 142.9]) # Work hours limit (max time over the period) for each person p
model.l = Param(model.t, initialize=[4.02, 5.1, 3.6, 4.29, 6.08]) # Duration of each task t in hours. Assumed to be independent on machine m!
model.d = Param(model.t, initialize=[124.02, 125.1, 123.6, 124.29, 126.08]) # Deadline of each task t in hours
model.r = Param(model.t, initialize=[123.02, 124.1, 122.6, 123.29, 125.08]) # Tardiness starts at these times for each task t
model.wc = Param(initialize=1.0) # Weight of cost objective
model.wt = Param(initialize=1.0) # Weight of tardiness objective

# Decision Variables
model.x = Var(model.m, within=NonNegativeReals, initialize=np.zeros(M)) # Time each machine m is used
model.y = Var(model.p, within=NonNegativeReals, initialize=np.zeros(P)) # Time each person p is employed
model.a = Var(model.t, model.m, domain=Binary); # Assignment of each task t to each machine m
model.b = Var(model.t, model.p, domain=Binary); # Assignment of each task t to each person m
model.f = Var(model.t, within=PositiveReals, initialize=[np.inf]*T) # Finish time for each task t

# Constraints

# Each machine has a usage limit: x[m] <= u[m];
def machine_time_limit(model, m): # for any machine m
    return model.x[m] <= model.u[m]
model.c1 = Constraint(model.m, rule = machine_time_limit) #set, rule

# Each person has a work hours limit: y[p] <= v[p];
def work_time_limit(model, p): # for any person p
    return model.y[p] <= model.v[p]
model.c2 = Constraint(model.p, rule = work_time_limit)

# Each task is assigned to only one machine: sum(m in 1..M) (a[t,m]) == 1;
def task_one_machine(model, t): # for any task t
    return sum(model.a[t,m] for m in model.m) == 1
model.c3 = Constraint(model.t, rule = task_one_machine)

# Each task is assigned to only one person: forall(t in 1..T) { sum(p in 1..P) (b[t,p]) == 1; };
def task_one_person(model, t): # for any task t
    return sum(model.b[t,p] for p in model.p) == 1
model.c4 = Constraint(model.t, rule = task_one_person)

# Resource Workload. Match task duration with machine and personnel usage:
def machine_use_time(model, m):
    return model.x[m] == sum(model.a[t,m]*model.l[t] for t in model.t)
model.c5 = Constraint(model.m, rule = machine_use_time)

def person_use_time(model, p):
    return model.y[p] == sum(model.b[t,p]*model.l[t] for t in model.t)
model.c6 = Constraint(model.p, rule = person_use_time)

# Finish time for each task t:
def finish_time(model, t):
    # Assuming tasks are executed in ascending order of its number t on each machine!
    return model.f[t] == sum(model.a[t,m] * sum(model.l[t1] for t1 in range(t+1)) for m in model.m) # Non-convex!
    # Reformulate to make it linear in a:
    # Truth Table:
    # a[t,m]   a[t1,m]   x
    # 0        0         0
    # 0        1         0
    # 1        0         0
    # 1        1         1
    #return model.f[t] == sum(sum(max(0, model.a[t,m] + model.a[t1,m] - 1) * model.l[t1] for t1 in range(t)) for m in model.m)
model.c7 = Constraint(model.t, rule = finish_time)
model.c7.pprint()

# Due Date for each task t: f[t] <= d[t];
def deadline_task(model, t): # for any task t
    return model.f[t] <= model.d[t]
model.c8 = Constraint(model.t, rule = deadline_task)

# Samples of constraints:
#model.c1 = model.Constraint(expr=model.q == 1)
#model.c2 = model.Constraint(expr=sum(model.y[i] for i in model.i) >= 2)

# Objective 1: Minimize the total cost of quality control resource allocation.
# Objective 2: Minimize the total tardiness.
model.objective = Objective(expr = 
    model.wc * (sum(model.c[m]*model.x[m] for m in model.m) + sum(model.e[p]*model.y[p] for p in model.p) + model.o) +
    model.wt * sum(model.f[t] - model.r[t] for t in model.t),                            
    sense=minimize)

c7 : Size=5, Index=t, Active=True
    Key : Lower : Body                                                                                       : Upper : Active
      0 :   0.0 :                                           f[0] - (4.02*a[0,0] + 4.02*a[0,1] + 4.02*a[0,2]) :   0.0 :   True
      1 :   0.0 :                                           f[1] - (9.12*a[1,0] + 9.12*a[1,1] + 9.12*a[1,2]) :   0.0 :   True
      2 :   0.0 : f[2] - (12.719999999999999*a[2,0] + 12.719999999999999*a[2,1] + 12.719999999999999*a[2,2]) :   0.0 :   True
      3 :   0.0 : f[3] - (17.009999999999998*a[3,0] + 17.009999999999998*a[3,1] + 17.009999999999998*a[3,2]) :   0.0 :   True
      4 :   0.0 : f[4] - (23.089999999999996*a[4,0] + 23.089999999999996*a[4,1] + 23.089999999999996*a[4,2]) :   0.0 :   True


In [516]:
# Solve
#solver = SolverFactory("ipopt.exe")
solver = SolverFactory('glpk') #GLPK Integer Optimizer 5.0

results = solver.solve(model, tee=True)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write C:\Users\Admin\AppData\Local\Temp\tmpitk672_8.glpk.raw --wglp C:\Users\Admin\AppData\Local\Temp\tmpvah4jc0n.glpk.glp
 --cpxlp C:\Users\Admin\AppData\Local\Temp\tmpak5czm3b.pyomo.lp
Reading problem data from 'C:\Users\Admin\AppData\Local\Temp\tmpak5czm3b.pyomo.lp'...
C:\Users\Admin\AppData\Local\Temp\tmpak5czm3b.pyomo.lp:284: warning: lower bound of variable 'x8' redefined
C:\Users\Admin\AppData\Local\Temp\tmpak5czm3b.pyomo.lp:284: warning: upper bound of variable 'x8' redefined
35 rows, 48 columns, 110 non-zeros
35 integer variables, all of which are binary
319 lines were read
Writing problem data to 'C:\Users\Admin\AppData\Local\Temp\tmpvah4jc0n.glpk.glp'...
245 lines were written
GLPK Integer Optimizer 5.0
35 rows, 48 columns, 110 non-zeros
35 integer variables, all of which are binary
Preprocessing...
10 rows, 35 columns, 35 non-zeros
35 integer variables, all of which are binary
Scaling...
 A: min|a

In [519]:
model.f.pprint() # print optimized model variable

f : Size=5, Index=t
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      0 :     0 :  4.02 :  None : False : False : PositiveReals
      1 :     0 :  9.12 :  None : False : False : PositiveReals
      2 :     0 : 12.72 :  None : False : False : PositiveReals
      3 :     0 : 17.01 :  None : False : False : PositiveReals
      4 :     0 : 23.09 :  None : False : False : PositiveReals


In [517]:
print("Print values for each variable explicitly")
for m in model.x:
    print(str(model.x[m]), model.x[m].value)
for p in model.y:
    print(str(model.y[p]), model.y[p].value)
for t in model.t:
    for m in model.x:
        print(str(model.a[t,m]), model.a[t,m].value)
    for p in model.p:
        print(str(model.b[t,p]), model.b[t,p].value)
    print(str(model.f[t]), model.f[t].value)

Print values for each variable explicitly
x[0] 0.0
x[1] 0.0
x[2] 23.09
y[0] 0.0
y[1] 0.0
y[2] 0.0
y[3] 23.09
a[0,0] 0.0
a[0,1] 0.0
a[0,2] 1.0
b[0,0] 0.0
b[0,1] 0.0
b[0,2] 0.0
b[0,3] 1.0
f[0] 4.02
a[1,0] 0.0
a[1,1] 0.0
a[1,2] 1.0
b[1,0] 0.0
b[1,1] 0.0
b[1,2] 0.0
b[1,3] 1.0
f[1] 9.12
a[2,0] 0.0
a[2,1] 0.0
a[2,2] 1.0
b[2,0] 0.0
b[2,1] 0.0
b[2,2] 0.0
b[2,3] 1.0
f[2] 12.72
a[3,0] 0.0
a[3,1] 0.0
a[3,2] 1.0
b[3,0] 0.0
b[3,1] 0.0
b[3,2] 0.0
b[3,3] 1.0
f[3] 17.01
a[4,0] 0.0
a[4,1] 0.0
a[4,2] 1.0
b[4,0] 0.0
b[4,1] 0.0
b[4,2] 0.0
b[4,3] 1.0
f[4] 23.09


In [527]:
model.x.pprint()

x : Size=3, Index=m
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      0 :     0 :   0.0 :  None : False : False : NonNegativeReals
      1 :     0 :   0.0 :  None : False : False : NonNegativeReals
      2 :     0 : 23.09 :  None : False : False : NonNegativeReals


In [541]:
print("Assignment of tasks to personnel:")
print("Task\tP1\tP2\tP3\tP4")
for t in range(T):
    print(t, end='');
    for p in range(P):
        print("\t",model.b[t,p].value, end='')
    print("")

Assignment of tasks to personnel:
Task	P1	P2	P3	P4
0	 0.0	 0.0	 0.0	 1.0
1	 0.0	 0.0	 0.0	 1.0
2	 0.0	 0.0	 0.0	 1.0
3	 0.0	 0.0	 0.0	 1.0
4	 0.0	 0.0	 0.0	 1.0


In [542]:
print("Assignment of tasks to machines:")
print("Task\tM1\tM2\tM3")
for t in range(T):
    print(t, end='');
    for m in range(M):
        print("\t",model.a[t,m].value * model.f[t].value, end='')
    print("")

Assignment of tasks to machines:
Task	M1	M2	M3
0	 0.0	 0.0	 4.02
1	 0.0	 0.0	 9.12
2	 0.0	 0.0	 12.72
3	 0.0	 0.0	 17.01
4	 0.0	 0.0	 23.09


In [543]:
print("Print values for all variables")
for v in model.component_data_objects(Var):
    print(str(v), v.value)

Print values for all variables
x[0] 0.0
x[1] 0.0
x[2] 23.09
y[0] 0.0
y[1] 0.0
y[2] 0.0
y[3] 23.09
a[0,0] 0.0
a[0,1] 0.0
a[0,2] 1.0
a[1,0] 0.0
a[1,1] 0.0
a[1,2] 1.0
a[2,0] 0.0
a[2,1] 0.0
a[2,2] 1.0
a[3,0] 0.0
a[3,1] 0.0
a[3,2] 1.0
a[4,0] 0.0
a[4,1] 0.0
a[4,2] 1.0
b[0,0] 0.0
b[0,1] 0.0
b[0,2] 0.0
b[0,3] 1.0
b[1,0] 0.0
b[1,1] 0.0
b[1,2] 0.0
b[1,3] 1.0
b[2,0] 0.0
b[2,1] 0.0
b[2,2] 0.0
b[2,3] 1.0
b[3,0] 0.0
b[3,1] 0.0
b[3,2] 0.0
b[3,3] 1.0
b[4,0] 0.0
b[4,1] 0.0
b[4,2] 0.0
b[4,3] 1.0
f[0] 4.02
f[1] 9.12
f[2] 12.72
f[3] 17.01
f[4] 23.09
